Reference

- https://github.com/adapter-hub/adapters
- https://docs.adapterhub.ml/loading.html
- https://docs.adapterhub.ml/adapter_composition.html

In [1]:
!pip install -q -U transformers

ERROR: adapter-transformers 3.2.1 has requirement tokenizers!=0.11.3,<0.14,>=0.11.1, but you'll have tokenizers 0.15.0 which is incompatible.
ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement urllib3==1.26.15, but you'll have urllib3 2.1.0 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: google-auth 2.22.0 has requirement urllib3<2.0, but you'll have urllib3 2.1.0 which is incompatible.


In [ ]:
!pip install accelerate

     |████████████████████████████████| 266kB 7.8MB/s eta 0:00:01
     |████████████████████████████████| 317kB 18.1MB/s eta 0:00:01
     |████████████████████████████████| 737kB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 62.8MB/s eta 0:00:01
     |████████████████████████████████| 286kB 91.5MB/s eta 0:00:01
     |██████████████▎                 | 298.0MB 200.5MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████▎   | 592.3MB 211.5MB/s eta 0:00:01

     |████████████████████████████▊   | 602.2MB 211.5MB/s eta 0:00:01

     |█████████████████████████████▎  | 612.2MB 211.5MB/s eta 0:00:01

...

     |███████████████████████████████ | 650.6MB 211.5MB/s eta 0:00:01

     |███████████████████████████████▌| 659.4MB 211.5MB/s eta 0:00:01

     |████████████████████████████████| 668.7MB 211.5MB/s eta 0:00:01

     |████████████████████████████████| 670.2MB 211.5MB/s 


     |████████████████████████████████| 61kB 82.4MB/s eta 0:00:01
     |████████████████████████████████| 174kB 95.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 100.3MB/s ta 0:00:01
     |████████████████████████████████| 81kB 102.0MB/s ta 0:00:01
     |▋                               | 430kB 97.6MB/s eta 0:00:01

In [8]:
!pip install bitsandbytes

     |████████████████████████████████| 92.6MB 7.6MB/s eta 0:00:01


In [2]:
!pip install -U adapters

     |████████████████████████████████| 235kB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 7.9MB 20.5MB/s eta 0:00:01
     |████████████████████████████████| 737kB 114.5MB/s eta 0:00:01
     |████████████████████████████████| 778kB 117.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 102.9MB/s ta 0:00:01
     |████████████████████████████████| 1.3MB 110.3MB/s eta 0:00:01
     |████████████████████████████████| 317kB 117.2MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 105.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 87.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 97.6MB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 115.7MB/s eta 0:00:01
     |████████████████████████████████| 174kB 109.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 79.8MB/s eta 0:00:01
     |████████████████████████████████| 163kB 109.9MB/s eta 0:00:01
     |████████████████████████████████| 143kB 112.6MB/s et

In [3]:
from adapters import AutoAdapterModel
from transformers import AutoTokenizer

In [6]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
import pandas as pd
import torch

In [9]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [ ]:
model = AutoAdapterModel.from_pretrained("roberta-base")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model.load_adapter("AdapterHub/roberta-base-pf-imdb", source="hf", set_active=True)

print(model(**tokenizer("This works great!", return_tensors="pt")).logits)